# Flask: templates et mise en forme 

[Vidéo d'accompagnement](https://vimeo.com/503856059)

Pour éviter de reprendre la partie sur sqlite, voici quelques fonctions pour nous simplifier la vie. Lire puis exécuter:

In [ ]:
import sqlite3 as db

def init_db():
    conn = db.connect("mini_blog.db") # connect(nom_fichier) renvoie un handler de connexion
    with open("schema.sql") as script: # Ouverture du fichier contenant le schéma de la base
        conn.executescript(script.read()) # Sa lecture donne une chaîne qu'on passe à executescript(sql)
    conn.close()

def recuperer_articles():
    """Renvoie une liste de tuples
    de la forme (id_article, login, cree_le, titre, contenu)"""
    
    with db.connect("mini_blog.db") as c:
        curs = c.execute(
            """SELECT articles.id, login, cree_le, titre, contenu 
               FROM membres JOIN articles
                 ON membres.id = articles.auteur_id
                 ORDER BY articles.cree_le DESC
            """
        )
        return curs.fetchall()

def inserer_article(auteur_id, titre, contenu):
    with db.connect("mini_blog.db") as c:
        curs = c.execute(
                 """INSERT INTO articles (auteur_id, titre, contenu) 
                   VALUES (?, ?, ?)""",
                (auteur_id, titre, contenu)
            )
        c.commit()
        
def supprimer_article(id_article):
    with db.connect("mini_blog.db") as c:
        curs = c.execute(
                 """DELETE FROM articles WHERE id = ?""",
                (id_article,)
            )
        c.commit()

def recuperer_membre(login):
    """Renvoie un tuple de la forme (id_membre, mot de passe)
    ou None si aucun membre n'a le login indiqué."""
    
    with db.connect("mini_blog.db") as c:
        curs = c.execute(
                 """SELECT id, mdp FROM membres WHERE login = ?""",
                (login,)
            )
        return curs.fetchone()

init_db()

## 6 - Introduction aux «templates»

Il n'est pas commode de mélanger le code Python avec le langage HTML. C'est la raison pour laquelle Flask nous propose d'utiliser les **templates** *jinja*.

> *werkzeug, jinja* ??? Flask est en réalité composé de plusieurs briques logicielles dédiées au web et groupées dans le projet [*pallets*](https://palletsprojects.com/).

L'idée d'un **template** est de pouvoir mélanger des éléments HTML avec une sorte de *mini-langage* dans le but de produire le HTML final. L'objectif est ici de **séparer** (*découpler*) le code python de l'application avec la description de la page que ce code vise à produire, en application du sacro-saint principe de **modularité** (découper le complexe en éléments plus simples ou modules).

Concrètement, un template jinja est une sorte de HTML avec des points d'insertions de la forme `{% instruction %}`. ou `{{ variable }}`. Par exemple pour écrire le HTML de notre page d'accueil précédente, on pourrait écrire un templates comme celui-ci:

```jinja2
{% for _, login, cree_le, titre, contenu in articles %}
    <article>
      <h3 style="margin-bottom: 0">{{ titre }}</h3>
      <p style="margin-top: 0; font-size: 80%">
      crée le {{ cree_le }} par <em>{{ login }}</em>
      </p>
      <p style="padding-left: 20px">{{ contenu }}</p>
      <hr/>
    </article>
{% endfor %}
```

Reprenons *app5* mais juste la page d'accueil. On commence par importer `render_template(nom_templates[, nom1=v1, nom2=v2, ..])` et son code devient (juste pour l'accueil):

In [ ]:
from flask import Flask, render_template

app6 = Flask(__name__, template_folder='templates/app6') # similaire à app5 mais avec l'utilisation des templates

@app6.route('/')
def accueil(): # comparer avec le code de app5
    return render_template(
        'accueil.html',           # nom du template (qui doit-être situé dans un dossier templates)
         articles=recuperer_articles() # variable «articles» accessible depuis le template
    )

In [ ]:
from serveur import lancer # lancer(application) ou lancer(application, port) 

lancer(app6, 8006)

Soyez vigilent aux éléments de syntaxe de ce mini-langage qui *ressemble* à Python mais n'**est pas Python**: il a ses propres règles...

Par exemple, il est nécessaire d'indiquer `{% endfor %}` car ce mini-langage est insensible à l'indentation. `{{valeur}}` NON mais `{{ valeur }}` est correct etc. En cas de doute, il faudra vous référerez à sa [documentation](https://jinja.palletsprojects.com/en/2.11.x/templates/).

## 7 - Composition de templates

Souvent, les multiples pages qui composent un site web dynamique ont une structure commune.

Par exemple, pour notre «Micro-blog NSI», on pourrait avoir le template de base (dans *templates/base.html*):

```jinja2
<!doctype html>
<html>
  <head>
    {% include 'head.html' %}
    <title>{% block titre %}{% endblock %}Micro Blog NSI</title>
  </head>
  <body>
    <nav>
      {% include 'nav.html' %}
    </nav>
    <section>
      <header>
        {% block entete %}{% endblock %}
      </header>
      {% block contenu %}{% endblock %}
    </section>
  </body>
</html>
```

Observez bien les `include` et les `block`. 

Les premiers - `include` - servent à **inclure** d'autres templates dans celui-ci: le résultat est simplement l'insertion du template «cible» dans celui-ci (le template «source»).

*Exemple* pour *nav.html* qui sera inclus dans *base.html*

```jinja2
<h1>Mini-Blog</h1>
<ul>
{% if 'login' in session %}
  <li><span>{{ session['login'] }}</span>
  <li><a href="{{ url_for('logout') }}">Se déconnecter</a>
{% else %}
  <li><a href="{{ url_for('inscription') }}">S'inscrire</a>
  <li><a href="{{ url_for('login') }}">Se connecter</a>
{% endif %}
</ul>
```

Les seconds - `block` - servent à créer de nouveaux templates par **spécialisation**: pour cela on crée un nouveau templates en commençant par `{% extends "base.html %}` (je spécialise le template «base.html») puis, on précise le contenu particulier des `block` (tout ou partie).

Ainsi un template qui étend *base.html* sera de la forme:

```jinja2
{% extends 'base.html' %}

{% block entete %}
...
{% endblock %}

{% block contenu %}
...
{% endblock %}
```

*Exemple*: *accueil.html* spécialise (étend) *base.html*

```jinja2
{% extends 'base.html' %}

{% block entete %}
  <h1>{% block titre %}Accueil{% endblock %}</h1>
  {% if 'mbrid' in session %}
    <a class="action" href="{{ url_for('editer') }}">Ajouter</a>
  {% endif %}
{% endblock %}

{% block contenu %}
    {% for article_id, login, cree_le, titre, contenu in articles %}
        <article>
          <head>
            <h1>{{ titre }}</h1>
            <aside>
              crée le {{ cree_le }} par <em>{{ login }}</em>
            </aside>
          </head>
          <p class="contenu">{{ contenu }}</p>
          {% if 'login' in session and session['login'] == login %}
            <a href="{{ url_for('supprimer', id=article_id) }}">supprimer cet article</a>
          {% endif %}
          {% if not loop.last %}
          <hr/>
          {% endif %}
        </article>
    {% endfor %}
{% endblock %}
```

Cet exemple est un peu long mais il vous permet d'observer que Flask transmet automatiquement aux templates l'objet `session` et la fonction `url_for`.

Observer encore que si une vue dépend d'un paramètre comme `supprimer(id_article)` (voir plus bas), on peut utiliser la syntaxe `url_for('nom_vue', nom_param=valeur)` pour créer l'url correspondante.

L'application Flask qui suit permet:
- la connexion et la déconnexion d'un membre,
- la suppression de ses articles pour un membre connecté.

In [ ]:
from flask import Flask, render_template, request, session, redirect, url_for
# from werkzeug.security import check_password_hash

app7 = Flask(__name__, template_folder='templates/app7')
app7.secret_key = "dev"

@app7.route('/')
def accueil():
    return render_template(
        'accueil.html', # template situé dans le répertoire templates/app7
        articles=recuperer_articles()
    )

@app7.route('/login', methods=["GET", "POST"])
def login():
    if request.method == "POST":
        login = request.form["login"]
        mdp = request.form["mdp"]
        membre = recuperer_membre(login)
        # membre = None ou est un 2-tuple de la forme (id, mdp)
        if membre and membre[1] == mdp: # mieux: check_password_hash(membre[1], mdp)
            session.clear()
            # enregistrons quelques informations utiles dans l'objet session.
            session['mbrid'] = membre[0]
            session['login'] = login
            return redirect(url_for('accueil'))
    # si "GET" ou si l'utilisateur n'est pas reconnue
    return render_template(
        'login.html'
    )

@app7.route('/logout')
def logout():
    session.clear()
    return redirect(url_for('accueil'))

@app7.route('/supprimer/<int:id>')
def supprimer(id):
    supprimer_article(id)
    return redirect(url_for('accueil'))

# la suite sert juste de «bouche trou»
@app7.route('/inscription')
def inscription():
    return redirect(url_for('accueil'))

@app7.route('/editer')
def editer():
    return redirect(url_for('accueil'))

In [ ]:
from serveur import lancer # lancer(application) ou lancer(application, port) 

lancer(app7, 8006)

In [ ]:
# Pour réinitialiser la base de données si besoin...
init_db()

## 8 - CSS pour le style

Pour charger un fichier de style (*stylesheet*):
1. créer un fichier *style.css*,
2. le placer dans un dossier nommé **static** situé au même niveau que le fichier définissant l'application,
3. et utiliser un ordre de chargement `<link ...>` dans le template *head.html*. 

Pour la 3e étape, on peut utiliser `url_for` (*head.html*):

```jinja2
<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
```

Reste à produire le code CSS ... À notre niveau, on procède par *ajustements successifs* ce qui demande un peu d'organisation.

Malheureusement la version communautaire de PyCharm n'offre aucune aide pour css (saisie, coloration syntaxique etc.)  ... mais on peut utiliser VSCodium pour s'en sortir. 

Autre point important: il faudra demander au navigateur de **rafraîchir le cache** en utilisant `CTRL+F5` (firefox) pour recharger la page; autrement, on ne voit pas les modifications apportées au niveau du style.

Voici une [vidéo qui vous montre comment procéder](https://vimeo.com/503790766):
- revoir les notions essentielles de css,
- organiser son environnement de travail,
- utiliser les outils de développement du navigateur pour
- procéder par ajustements successifs.

Exemple de code css de *style.css*:

```css
html {
    font-family: sans-serif;
    background: rgb(252, 255, 240); /* floralwhite */
}

body {
    max-width: 960px;
    margin: 0 auto;
    background: rgb(255, 253, 250);
    padding-bottom: 2rem;
}

nav {
    display: flex;
    height: 50px;
    justify-content: space-between;
    align-items: baseline;
    background: rgb(255, 238, 203);
}

nav h1 {
    padding-left: 1rem;
    margin: .5rem 0;
}

nav ul {
    display: flex;
    list-style-type: none;
}

nav ul li {
    margin-right: 1rem;
}

a {
    text-decoration: none;
}

section {
    padding-left: 1rem;
}

h1, a {
    color: rgb(73, 100, 187);
}

aside {
    font-size: smaller;
    color: rgb(122, 122, 122);
    margin-left: 0.3rem;
}

article h1 {
    margin-bottom: 0.5rem;
}

.contenu {
    padding: 0.5rem 2rem;
}

hr {
    width: 500px;
    border-bottom: 0;
    border-left: 0;
    border-right: 0;
    border-top: 1px solid lightgrey;
}

form {
    margin: 3rem auto;
    width: 500px;
    padding: 1rem;
    border: 1px solid #CCC;
    border-radius: 1rem;
}

form label {
    display: inline-block;
    width: 35%;
    text-align: right;
    margin-bottom: 1rem;
    font-weight: bold;
}

form div:last-child {
    margin-top: 1em;
    text-align: center;
}
```